In [2]:
import sys, os
sys.path.append('/scratch/oem214/vanilla-rtrl/')
from cluster import submit_jobs
from continual_learning import *
from core import *
from dynamics import *
from functions import *
from gen_data import *
from learning_algorithms import *
from optimizers import *
from plotting import *
from wrappers import *

In [6]:
### --- SET UP ALL CONFIGS --- ###
from itertools import product
n_seeds = 10
macro_configs = config_generator(lr=[0.01, 0.001],
                                 L2_reg=[0.001, 0.0001])
micro_configs = tuple(product(macro_configs, list(range(n_seeds))))

### --- SELET PARTICULAR CONFIG --- ###
try:
    i_job = int(os.environ['SLURM_ARRAY_TASK_ID']) - 1
except KeyError:
    i_job = 0
params, i_seed = micro_configs[i_job]
i_config = i_job//n_seeds

new_random_seed_per_condition = True
if new_random_seed_per_condition:
    np.random.seed(i_job)
else: #Match random seeds across conditions
    np.random.seed(i_seed)

In [7]:
task = Flip_Flop_Task(3, 0.05, input_magnitudes=None)
N_train = 20000
N_test = 5000
checkpoint_interval = 100
sigma = 0

data = task.gen_data(N_train, N_test)

n_in = task.n_in
n_hidden = 32
n_out = task.n_out
W_in  = np.random.normal(0, np.sqrt(1/(n_in)), (n_hidden, n_in))
W_rec = np.random.normal(0, np.sqrt(1/n_hidden), (n_hidden, n_hidden))
W_out = np.random.normal(0, np.sqrt(1/(n_hidden)), (n_out, n_hidden))

b_rec = np.zeros(n_hidden)
b_out = np.zeros(n_out)

alpha = 1
sigma = 0

rnn = RNN(W_in, W_rec, W_out, b_rec, b_out,
          activation=tanh,
          alpha=alpha,
          output=identity,
          loss=mean_squared_error)

learn_alg = RFLO(rnn, alpha=alpha, L2_reg=params['L2_reg'], L1_reg=0.0001)
optimizer = SGD_Momentum(lr=params['lr'], mu=0.6)

monitors = []

sim = Simulation(rnn)
sim.run(data, learn_alg=learn_alg, optimizer=optimizer,
        sigma=sigma,
        monitors=monitors,
        verbose=True,
        report_accuracy=False,
        report_loss=True,
        checkpoint_interval=checkpoint_interval)

Progress: 10.0% complete 
Time Elapsed: 0.5s 
Test loss: 0.5926875651320297 

Progress: 20.0% complete 
Time Elapsed: 1.1s 
Test loss: 0.22567720176092662 

Progress: 30.0% complete 
Time Elapsed: 1.8s 
Test loss: 0.17166667707741476 

Progress: 40.0% complete 
Time Elapsed: 2.5s 
Test loss: 0.013490525979990875 

Progress: 50.0% complete 
Time Elapsed: 3.1s 
Test loss: 0.00549117957887174 

Progress: 60.0% complete 
Time Elapsed: 3.8s 
Test loss: 0.04247337704855229 

Progress: 70.0% complete 
Time Elapsed: 4.4s 
Test loss: 0.0041677683038541715 

Progress: 80.0% complete 
Time Elapsed: 5.1s 
Test loss: 0.020775796418716158 

Progress: 90.0% complete 
Time Elapsed: 5.7s 
Test loss: 0.003726477232464915 



In [8]:
### --- SAVE RESULTS -- ###
result = {'sim': sim, 'i_seed': i_seed, 'task': task,
          'config': params, 'i_config': i_config, 'i_job': i_job}
try:
    result['processed_data'] = processed_data
except NameError:
    pass
    
try:
    save_dir = os.environ['SAVEDIR']
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_path = os.path.join(save_dir, 'result_'+str(i_job))

    with open(save_path, 'wb') as f:
        pickle.dump(result, f)
except KeyError:
    pass

In [10]:
os.path.join('hell', 'potato', 'wants', 'you', 'dead')

'hell/potato/wants/you/dead'

In [5]:
###Truncate file above
file_name = 'pipeline_test'
get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.save_notebook()')
get_ipython().system('jupyter nbconvert --to script --no-prompt {}.ipynb'.format(file_name))
get_ipython().system('awk "/###Truncate/ {{exit}} {{print}}" {}.py'.format(file_name))
get_ipython().system('sed -i "/###Truncate/Q" {}.py'.format(file_name))
get_ipython().system('mv {}.py ../cluster_main_scripts'.format(file_name))

<IPython.core.display.Javascript object>

[NbConvertApp] Converting notebook pipeline_test.ipynb to script
[NbConvertApp] Writing 1500 bytes to pipeline_test.py
awk: 1: unexpected character '.'


In [ ]:
###Submit job to cluster
job_name = 'scratch'
n_jobs = len(micro_configs)
write_job_file(job_name, py_file_name='{}.py'.format(file_name))
submit_job()